# Пример из исследования: подготовка данных данных ссуд для самообслуживания BI

<b> замечание: </b> для этого тематического исследования мы будем использовать Hortonbox (версия была 2.3.2 на момент написания этой главы)

### Чем заняться на Hortonbox до работы системы:

<p> <b> замечание: </b> вы можете подключиться к Hortonbox через PuTTY, это обычно проще, чем работа над самой виртуальной машиной </p>
<p> yum -y установить python -pip </p>
<p> PIP установить git+https: //github.com/davycielen/pywebhdfs.git -облегчить </p>
<p> <b> замечание: </b> Этот пользовательский репозиторий представляет собой вилку pywebhdfs. Pywebhdfs был сломлен на момент написания статьи. Реквизит для его исправления был выпущен, но пока основной репо снова не сработает должным образом, используйте пользовательский, который мы создали </p>
<p> pip установить панды </p>

<p> <b> замечание: </b> все следующее <b> код </b> необходимо запустить <b> в саме Hortonbox </b> </p>
<p> Чтобы сделать это, откройте сессию замазки и выпустите команду «Pyspark», это откроет интерпретатор Python </p>
<p> Если все прошло уже задолго до этого, этот переводчик также должен иметь библиотеки Pandas и pywebhdfs (или адаптированных Pywebhdfs) </p>
<p> Весь следующий код должен быть запущен в этом сеансе Pyspark (вы можете просто скопировать вставку его кусочком) </p>
<p> Существуют другие параметры, такие как копирование кода в файл .py и запуск его там, другой вариант - ноутбук Zeppelin </p>
<p> Zeppelin установлен на Hortonbox, но не был достаточно зрелым, чтобы полагаться на момент написания. </p>

## Часть 1 процесса науки о данных: цель исследования

<b> Основная цель обучения: </b> Использование технологий больших данных для подготовки данных для <b> самообслуживания Bi </b>. В этом тематическом исследовании мы не пойдем в построение модели, а вместо этого тратим больше времени на подготовку данных, чтобы позволить другим людям исследовать его и в конечном итоге создавать отчеты, используя подготовленные данные. Концепция BI самообслуживания позволяет конечным пользователям находить свои собственные идеи и часто применяется, когда компания очень ориентирована на данные, но не хватает ученых данных, чтобы предоставить информацию. В главе 9 также будет представлена ​​тематическое исследование с некоторым более вниманием на применении уровня панели панели для самообслуживания BI.

## Часть 2 процесса данных: поиск данных

### Загрузите данные

In [ ]:
try:
    from StringIO import StringIO # # для Python 2
except ImportError:
    from io import StringIO # # для Python 3

In [6]:
import requests
import zipfile
# импортировать Stringio

In [12]:
source = requests.get("https://resources.lendingclub.com/LoanStats3d.csv.zip", verify=False) # А

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'resources.lendingclub.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [ ]:
print(source.content)

Buffered data was truncated after reaching the output size limit.

In [14]:
stringio = StringIO(source) # Беременный
unzipped = zipfile.ZipFile(stringio) # В

TypeError: initial_value must be str or None, not Response

In [ ]:
# Загрузите данные из кредитного клуба. Это HTTPS, так что это должно быть проверено, но мы не будем беспокоиться (проверьте = false)
# B создает виртуальный файл.
# C Разанипируйте данные

### Переместить данные в Hadoop

In [ ]:
import pandas as pd
from pywebhdfs.webhdfs import PyWebHdfsClient

In [ ]:
subselection_csv = pd.read_csv(unzipped.open('LoanStats3d.csv'),skiprows=1,skipfooter=2,engine='python') # А
stored_csv = subselection_csv.to_csv('./stored_csv.csv') # Беременный

In [ ]:
hdfs = PyWebHdfsClient(user_name="hdfs",port=50070,host="sandbox")# В
hdfs.make_dir('chapter5')# Дюймовый
with open('./stored_csv.csv') as file_data: # Эн
        hdfs.create_file('chapter5/LoanStats3d.csv',file_data, overwrite=True)# Фон

In [ ]:
# A Сделайте некоторую предварительную очистку данных с помощью Pandas: удалить верхний ряд и нижние 2 ряда, потому что они бесполезны для нас. Просто открытие оригинального файла покажет вам это.
# B Храните его локально, потому что нам нужно перенести его в файловую систему Hadoop
# C Подключитесь к песочнице Hadoop
# D Создать папку с названием «Глава 5» на файловой системе Hadoop
# E Откройте локально хранящийся CSV
# F Создайте файл .csv на файловой системе Hadoop

In [ ]:
print hdfs.get_file_dir_status('chapter5/LoanStats3d.csv')# А

In [ ]:
# Распечатать статус файла

In [ ]:
# hdfs.delete_file_dir ('Глава 5/Loanstats3d.csv', recurisive = true) #A

In [ ]:
# Код на случай, если вы хотите удалить файл

## Часть 3 процесса данных: подготовка данных

### Очистка данных Apache Spark

In [ ]:
from pyspark import SparkContext # А
from pyspark.sql import HiveContext # Беременный
# sc = sparkcontext ()#c
sqlContext = HiveContext(sc)# Дюймовый
data = sc.textFile("/chapter5/LoanStats3d.csv") # Эн
parts = data.map(lambda r:r.split(',')) # Фон
firstline = parts.first() # Глин
datalines = parts.filter(lambda x:x != firstline)# ЧАС
def cleans(row): # я
        row[7] = str(float(row[7][:-1])/100)# Дж
        return [s.encode('utf8').replace(r"_"," ").lower() for s in row]# K

In [ ]:
datalines = datalines.map(lambda x: cleans(x))# Л

In [ ]:
# Контекст импорта искры -> не обязательно при работе непосредственно в Pyspark!
# B Контекст импорта улей
# C В сеансе Pyspark SparkContext автоматически присутствует. В других случаях (ноутбук Zeppelin) вам нужно будет
# Создайте это явно
# D Создать HiveContext
# E загрузка в наборе данных из каталога Hadoop
# F Разделите набор данных с разделителем komma (,). Это концом разделителя строки для этого файла
# G Возьмите первую строку
# H Храните все линии, но первая строка. Потому что первая строка - это просто имена переменных
# I Функция очистки будет использовать силу искры для очистки данных. Ввод этой функции будет линейкой данных.
# J столбец 8 (index = 7) имеет % форматированных чисел. Нам не нужен этот знак «%».
# K кодирует все в UTF8, замените подчеркивание на пробелы и все строчные.
# L Экскурация линии очистки данных по линии

### Сохранение данных в Hive

In [ ]:
from pyspark.sql.types import * # А
fields = [StructField(field_name,StringType(),True) for field_name in firstline] # Беременный
schema = StructType(fields) # В
schemaLoans = sqlContext.createDataFrame(datalines, schema) # Дюймовый
schemaLoans.registerTempTable("loans") # Эн

sqlContext.sql("drop table if exists LoansByTitle") # Фон
sql = '''create table LoansByTitle stored as parquet as select title, count(1) as number from loans group by title order by number desc'''# Фон
sqlContext.sql(sql)# Фон

sqlContext.sql('drop table if exists raw') # Глин
sql = '''create table raw stored as parquet as select title, emp_title,grade,home_ownership,int_rate,recoveries,collection_recovery_fee,loan_amnt,term from loans'''# Глин
sqlContext.sql(sql)# Глин

In [ ]:
# Импорт типов данных SQL
# B Создание метаданных: функция Spark SQL Structfield представляет поле в структуре.
# Объект Structfield состоит из трех полей: name (a String), DataType (DataType) и «Nullable» (логический).
# Поле имени является названием Structfield.
# Поле DataType определяет тип данных Structfield.
# Поле Nullable указывает, могут ли значения структуры не содержать значений.
# C Functype Функция создает схему данных. Объект structtype требует списка структуры в качестве Imput.
# D Создайте DataFrame из Data (DataLines) и Dataschema (Schema)
# E Зарегистрируйте его в виде таблицы, называемого кредитами
# F Drop Table (в случае, если она уже существует), суммируйте и храните в Hive. Loansbytitle представляет сумму кредитов по названию работы
# G Drop Table (в случае, если она уже существует) и сохранить подмножество необработанных данных в Hive

## Часть 4 процесса данных: исследование данных и часть 6: презентация для конечного пользователя

Разведка данных, часть этого тематического исследования, будет сделана в Qlick. В этом тематическом исследовании нет реальной фазы моделирования. Поскольку Qlick позволяет вам создавать панель панели и отчеты, часть 4 и 6 образуют перекрытие.